In [9]:
# ============================
# LEGAL RAG BOOTSTRAP CELL
# ============================

from dotenv import load_dotenv
import os
import json
import pickle
import faiss
from sentence_transformers import SentenceTransformer

# Load environment variables
load_dotenv()
assert os.getenv("GROQ_API_KEY") is not None, "❌ GROQ_API_KEY not found"

# Load embedding model (ONLY ONCE)
embedding_model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device="cpu"
)

# Load FAISS index
index = faiss.read_index("vector-store/faiss.index")

# Load metadata
with open("vector-store/metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Load chunk texts
with open("processed-data/judgment_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

texts = [c["text"] for c in chunks]

# Retrieval function
def retrieve_precedents(query, top_k=5):
    query_embedding = embedding_model.encode(
        [query],
        convert_to_numpy=True
    )

    distances, indices = index.search(query_embedding, top_k)

    return [
        {
            "case_name": metadata[idx]["case_name"],
            "chunk_id": metadata[idx]["chunk_id"],
            "text": texts[idx][:400]
        }
        for idx in indices[0]
    ]

print("✅ Bootstrap complete — retrieval ready")


✅ Bootstrap complete — retrieval ready


In [14]:
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
MODEL_NAME = "llama-3.3-70b-versatile"

print("✅ Groq LLM initialized")


✅ Groq LLM initialized


In [11]:
def build_precedent_prompt(query, retrieved_chunks):
    context = "\n\n".join(
        f"Case: {c['case_name']}\nExcerpt: {c['text']}"
        for c in retrieved_chunks
    )

    return f"""
You are a legal research assistant.

User Query:
"{query}"

Below are excerpts from relevant court judgments:

{context}

Task:
Explain why each judgment is relevant to the query.

Guidelines:
- Focus on legal principles and constitutional interpretation
- Explicitly cite each case
- Do NOT give legal advice
- Do NOT invent facts
- Use formal legal language
- Keep the explanation concise and structured
"""


In [5]:
def build_precedent_prompt(query, retrieved_chunks):
    context = "\n\n".join(
        f"Case: {c['case_name']}\nExcerpt: {c['text'][:800]}"
        for c in retrieved_chunks
    )

    return f"""
You are a legal research assistant.

User Query:
"{query}"

Below are excerpts from relevant court judgments:

{context}

Task:
Explain why each judgment is relevant to the query.

Guidelines:
- Focus on legal principles and constitutional interpretation
- Explicitly cite each case
- Do NOT give legal advice
- Do NOT invent facts
- Use formal legal language
- Keep the explanation concise and structured
"""


In [12]:
def explain_precedents(query, retrieved_chunks):
    prompt = build_precedent_prompt(query, retrieved_chunks)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content


In [7]:

def explain_precedents(query, retrieved_chunks):
    prompt = build_precedent_prompt(query, retrieved_chunks)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content


In [15]:
query = "procedure established by law under Article 21"

retrieved = retrieve_precedents(query, top_k=5)

result = explain_precedents(query, retrieved)

print(result)
query = "procedure established by law under Article 21"

retrieved = retrieve_precedents(query, top_k=5)

result = explain_precedents(query, retrieved)

print(result)




The query "procedure established by law under Article 21" pertains to the constitutional interpretation of the phrase "procedure established by law" in the context of the right to life and personal liberty guaranteed under Article 21 of the Constitution.

The judgment in **Justice K.S. Puttaswamy (Retd.) and Anr. vs. Union of India and Ors. (2017)** is relevant as it elucidates that the expression "procedure established by law" in Article 21 signifies a procedure that is not only present in enacted law but also fair, just, and reasonable (Justice K.S. Puttaswamy (Retd.) and Anr. vs. Union of India and Ors., 2017). This interpretation highlights the substantive aspect of the procedure, emphasizing that it must be of a certain quality to meet the constitutional mandate.

The judgment in **Maneka Gandhi vs. Union of India (1978)** is also pertinent as it reiterates that the procedure prescribed by law must be fair, just, and reasonable, and not arbitrary, fanciful, or oppressive (Maneka G